In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

NameError: name 'nvidia' is not defined

In [3]:
torch.cuda.device_count()

0

In [ ]:
torch.cuda.current_device()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the data
data = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\IQR.csv')
data = data[['Date', 'Price']]

# Preprocess the data
scaler = MinMaxScaler(feature_range=(-1, 1))
price_data = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

# Define the model
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the hyperparameters and instantiate the model
input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm.to(device)

# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)

# Train the model
for epoch in range(1000):
    inputs = Variable(torch.from_numpy(price_data[:-1]).float()).to(device)
    labels = Variable(torch.from_numpy(price_data[1:]).float()).to(device)

    optimizer.zero_grad()

    outputs = lstm(inputs)
    loss = criterion(outputs.view(-1), labels.view(-1))
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch} Loss: {loss.item()}')
